In [3]:
import statistics
import json
import pandas as pd
import re
import nltk
import numpy as np
import scipy.stats as stats
nltk.download('punkt')

import os
os.listdir('/kaggle/input/')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['indian-supreme-court-judgements']

In [2]:
file_path = '/kaggle/input/indian-supreme-court-judgements/data12.json'

# Read the JSON file
with open(file_path, 'r') as f:
    dictionaries = json.load(f)

# #Check one document
# print(dictionaries[0])

In [3]:
!pip install rouge_score sacrebleu torch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f76934dbec2739941101e8960d9989422d6cca8ecf7fecf0e87a7e820d6a755d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
from rouge_score import rouge_scorer
import sacrebleu
import torch

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer1 = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")
model1 = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus").to(device)

tokenizer2 = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")
model2 = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384").to(device)

tokenizer3 = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model3 = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6").to(device)

tokenizer4 = AutoTokenizer.from_pretrained("law-ai/InCaseLawBERT")
model4 = AutoModel.from_pretrained("law-ai/InCaseLawBERT").to(dev)

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/343 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/534M [00:00<?, ?B/s]

In [6]:
def precision_recall_f1(reference_summary, generated_summary):
  # Tokenize both summaries into sentences first, then words
  ref_sentences = nltk.sent_tokenize(reference_summary.lower())
  gen_sentences = nltk.sent_tokenize(generated_summary.lower())

  # Flatten sentence tokens into a single set of words
  ref_tokens = set(word for sentence in ref_sentences for word in nltk.word_tokenize(sentence))
  gen_tokens = set(word for sentence in gen_sentences for word in nltk.word_tokenize(sentence))

  # True Positives: Words in both reference and generated summaries
  true_positives = ref_tokens.intersection(gen_tokens)

  # Precision: True Positives / (True Positives + False Positives)
  precision = len(true_positives) / len(gen_tokens) if len(gen_tokens) > 0 else 0

  # Recall: True Positives / (True Positives + False Negatives)
  recall = len(true_positives) / len(ref_tokens) if len(ref_tokens) > 0 else 0

  # F1 Score: 2 * (Precision * Recall) / (Precision + Recall)
  f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

  return precision, recall, f1

In [7]:
def generate_summary(text, tokenizer, model):
    input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=1024,truncation=True).to(device)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=300,
                                  max_length=3000,
                                  early_stopping=True)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    # print(summary)
    return summary

def evaluate_summary(generated_summary, reference_summary, ref):
    
    reference = [nltk.word_tokenize(" ".join(reference_summary))]
    hypothesis =[generated_summary]
    ref1 = [nltk.word_tokenize(sentence) for sentence in ref]

    # BLEU score using sacrebleu
    bleu_score = sacrebleu.corpus_bleu(hypothesis, ref1).score

    # ROUGE score calculation
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge.score(reference_summary, generated_summary)

    precision, recall, f1 = precision_recall_f1(" ".join(reference_summary), generated_summary)

    return {
        "bleu": bleu_score,
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rouge2": rouge_scores['rouge2'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [4]:
import math
dataset = dictionaries[9600:10000]


# MODEL 1 - Legal-BERT

In [9]:
from transformers import BertTokenizer, BertModel
import torch

# Load the Legal-BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')
model = BertModel.from_pretrained('nlpaueb/legal-bert-base-uncased').to(device)

def get_legalbert_embeddings(sentences):
    embeddings = []
    
    for sentence in sentences:
        
        # Tokenize the sentence and convert to input IDs and attention masks
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()} 
        with torch.no_grad():
            outputs = model(**inputs)
        
        # The embeddings are the output of the last hidden state
        # We take the mean of the token embeddings to get a sentence embedding
        sentence_embedding = outputs.last_hidden_state.mean(dim=1).cpu().squeeze()
        embeddings.append(sentence_embedding)

    return torch.stack(embeddings)
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity_scores(embeddings, threshold=0.4):
    sentence_count = len(embeddings)
    counts = [0] * sentence_count  # Initialize a count for each sentence

    for i in range(sentence_count):
        for j in range(sentence_count):
            if i != j:
                similarity = cosine_similarity(
                    embeddings[i].unsqueeze(0), embeddings[j].unsqueeze(0)
                )[0][0]
                if similarity > threshold:
                    counts[i] += 1
    return counts
def select_top_sentences(sentences, counts, top_percent=0.4):
    # Zip sentences with their counts and sort based on counts
    sorted_results = sorted(zip(sentences, counts), key=lambda x: x[1], reverse=True)
    num_sentences = int(top_percent * len(sorted_results))  # Top 40%
    top_sentences = [item[0] for item in sorted_results[:num_sentences]]  # Select sentences
    return top_sentences
def generate_summary_from_top_sentences(top_sentences):
    return ' '.join(top_sentences)


def evaluate_summary(generated_summary, reference_summary, ref):
    # Tokenization for BLEU score calculation
    reference = [nltk.word_tokenize(reference_summary)]
    hypothesis = nltk.sent_tokenize(generated_summary)
    ref_tokenized = [nltk.word_tokenize(sentence) for sentence in ref]

    # BLEU score using sacrebleu
    bleu_score = sacrebleu.corpus_bleu(hypothesis, ref_tokenized).score

    # ROUGE score calculation
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge.score(" ".join(reference_summary), " ".join(generated_summary))
    
    precision, recall, f1 = precision_recall_f1(" ".join(reference_summary), generated_summary)


    # You can remove or replace precision/recall/F1 with more appropriate metrics if needed
    return {
        "bleu": bleu_score,
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rouge2": rouge_scores['rouge2'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure,
        "precision":precision,
        "recall":recall,
        "f1":f1
    }


def evaluate_dataset_app1(dataset, tokenizer, model):
    count=0
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
      }

    for doc in dataset:
        
        judgement_text = (doc['judgement_sent'])
        reference_summary = (doc['headnote_sent'])
        ref = doc['headnote_sent']

        
        sentence_embeddings = get_legalbert_embeddings(judgement_text)
    
        
        counts = compute_similarity_scores(sentence_embeddings, threshold=0.4)
    
        
        top_sentences = select_top_sentences(judgement_text, counts, top_percent=0.4)
    
        
        filtered_summary = generate_summary_from_top_sentences(top_sentences)
        generated_summary = generate_summary(filtered_summary, tokenizer, model)
    
        
        scores = evaluate_summary((generated_summary), " ".join(reference_summary), ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count+=1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [10]:
dataset = dictionaries[9600:10000]
# avg_scores_app1_3 = evaluate_dataset_app1(dataset, tokenizer3, model3)
avg_scores_app1_1= evaluate_dataset_app1(dataset, tokenizer1, model1)
# print(avg_scores_app1_3)
print(avg_scores_app1_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [11]:
avg_scores_app1_2= evaluate_dataset_app1(dataset, tokenizer2, model2)
# print(avg_scores_app1_3)
print(avg_scores_app1_2)

1 2 

Input ids are automatically padded from 328 to 1024 to be a multiple of `config.attention_window`: 1024


3 

Input ids are automatically padded from 648 to 1024 to be a multiple of `config.attention_window`: 1024


4 

...

382 383 

Input ids are automatically padded from 967 to 1024 to be a multiple of `config.attention_window`: 1024


384 385 386 387 388 389 390 391 

Input ids are automatically padded from 793 to 1024 to be a multiple of `config.attention_window`: 1024


392 393 394 395 396 397 398 399 400 
{'bleu': 0.3379454506332079, 'rouge1': 0.6407332161196361, 'rouge2': 0.56429232285195, 'rougeL': 0.3775536983373209, 'precision': 0.04204682670074229, 'recall': 0.17594008790632537, 'f1': 0.06767580072112395}


In [11]:
#WITHOUT Stop words
avg_scores_app1_3= evaluate_dataset_app1(dataset, tokenizer3, model3)
print(avg_scores_app1_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

# MODEL 2- TEXTRANK with TF-IDF

In [11]:
!pip install networkx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

def compute_sentence_similarity(sentences):
    # Use TF-IDF Vectorizer to convert sentences to vectors
    vectorizer = TfidfVectorizer().fit_transform(sentences)
    # Compute cosine similarity between sentence vectors
    similarity_matrix = cosine_similarity(vectorizer)
    return similarity_matrix


def rank_sentences_with_textrank(sentences, similarity_matrix):
    # Create a graph where each node is a sentence
    nx_graph = nx.from_numpy_array(similarity_matrix)

    # Apply the TextRank (PageRank) algorithm on the graph
    scores = nx.pagerank(nx_graph)

    # Rank sentences based on their TextRank score
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    return ranked_sentences


def select_top_sentences_textrank(ranked_sentences, top_percent=0.4):
    # Select the top X% of sentences
    num_sentences = int(top_percent * len(ranked_sentences))
    top_sentences = [sent[1] for sent in ranked_sentences[:num_sentences]]
    return top_sentences

def evaluate_dataset_app2(dataset, tokenizer, model):
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
    }
    count=0

    for doc in dataset:
        reference_summary = " ".join(doc['headnote_sent'])
        judgement =   doc['judgement_sent']
        ref = doc['headnote_sent']

        
        similarity_matrix = compute_sentence_similarity(judgement)

        
        ranked_sentences = rank_sentences_with_textrank(judgement, similarity_matrix)

        
        top_sentences = select_top_sentences_textrank(ranked_sentences, top_percent=0.4)

        
        generated_summary = generate_summary(" ".join(top_sentences), tokenizer, model)

        
        scores = evaluate_summary(generated_summary, reference_summary, ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count += 1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

In [12]:
avg_scores_app2_1= evaluate_dataset_app2(dataset, tokenizer1, model1)
print(avg_scores_app2_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [13]:
avg_scores_app2_2= evaluate_dataset_app2(dataset, tokenizer2, model2)
print(avg_scores_app2_2)

1 

Input ids are automatically padded from 433 to 1024 to be a multiple of `config.attention_window`: 1024


2 3 4 5 6 

Input ids are automatically padded from 904 to 1024 to be a multiple of `config.attention_window`: 1024


7 

...

360 361 362 363 364 365 366 367 368 369 370 371 372 373 

Input ids are automatically padded from 531 to 1024 to be a multiple of `config.attention_window`: 1024


374 

Input ids are automatically padded from 981 to 1024 to be a multiple of `config.attention_window`: 1024


375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 
{'bleu': 0.11830232069460188, 'rouge1': 0.40278215766410297, 'rouge2': 0.16913052520809285, 'rougeL': 0.20063932321023437, 'precision': 0.04233946804210546, 'recall': 0.1792594983378638, 'f1': 0.0683236386092253}


In [14]:
avg_scores_app2_3= evaluate_dataset_app2(dataset, tokenizer3, model3)
print(avg_scores_app2_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

# MODEL 3  - LSA

In [10]:
def compute_similarity_scores(embeddings, threshold=0.4):
    sentence_count = len(embeddings)
    counts = [0] * sentence_count  # Initialize a count for each sentence

    for i in range(sentence_count):
        for j in range(sentence_count):
            if i != j:
                similarity = cosine_similarity(embeddings[i].reshape(1, -1), embeddings[j].reshape(1, -1))[0][0]
                if similarity > threshold:
                    counts[i] += 1
    return counts

def select_top_sentences(sentences, counts, top_percent=0.4):
    sorted_results = sorted(zip(sentences, counts), key=lambda x: x[1], reverse=True)
    num_sentences = int(top_percent * len(sorted_results))  # Top 40%
    top_sentences = [item[0] for item in sorted_results[:num_sentences]]  # Select sentences
    return top_sentences

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

def compute_sentence_embeddings_lsa(sentences, n_components=100):
 
  vectorizer = TfidfVectorizer(max_features=5000)  
 
  tfidf_matrix = vectorizer.fit_transform(sentences)

  svd_model = TruncatedSVD(n_components=n_components, random_state=42)
  sentence_embeddings = svd_model.fit_transform(tfidf_matrix)

  return sentence_embeddings

In [12]:
def evaluate_dataset_app3(dataset, tokenizer, model):
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
    }
    count=0

    for doc in dataset:
        
        reference_summary = " ".join(doc['headnote_sent'])
        judgement =  doc['judgement_sent']
        ref = doc['headnote_sent']

        
        #sentence_embeddings = get_sentence_embeddings(judgement)
        sentence_embeddings = compute_sentence_embeddings_lsa(judgement)
        
        
        counts = compute_similarity_scores(sentence_embeddings, threshold=0.4)

        
        top_sentences = select_top_sentences(judgement, counts, top_percent=0.4)

       
        generated_summary = generate_summary(" ".join(top_sentences), tokenizer, model)

        
        scores = evaluate_summary(generated_summary, reference_summary, ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count += 1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

In [13]:
avg_scores_app3_1= evaluate_dataset_app3(dataset, tokenizer1, model1)
print(avg_scores_app3_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [18]:
avg_scores_app3_3= evaluate_dataset_app3(dataset, tokenizer3, model3)
print(avg_scores_app3_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [20]:
avg_scores_app3_2= evaluate_dataset_app3(dataset, tokenizer2, model2)
print(avg_scores_app3_2)

1 

Input ids are automatically padded from 295 to 1024 to be a multiple of `config.attention_window`: 1024


2 3 4 

Input ids are automatically padded from 942 to 1024 to be a multiple of `config.attention_window`: 1024


5 6 7 

...

376 377 378 379 380 381 

Input ids are automatically padded from 853 to 1024 to be a multiple of `config.attention_window`: 1024


382 383 384 385 386 387 388 389 390 391 

Input ids are automatically padded from 963 to 1024 to be a multiple of `config.attention_window`: 1024


392 393 394 395 396 397 398 399 400 
{'bleu': 0.12185958516028504, 'rouge1': 0.39105501180344154, 'rouge2': 0.15901209420031373, 'rougeL': 0.1964310185190708, 'precision': 0.04212494333145742, 'recall': 0.17464730749224283, 'f1': 0.06771428609874715}


# MODEL 4 - InCaseLawBERT

In [13]:
def get_sentence_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer4(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()} 
        with torch.no_grad():
            output = model4(**inputs)
        # Use the mean of the last hidden state as the sentence embedding
        embedding = output.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(embedding)
    return embeddings

In [14]:
def evaluate_dataset_app4(dataset, tokenizer, model):
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
    }
    count=0

    for doc in dataset:
        reference_summary = " ".join(doc['headnote_sent'])
        judgement =  doc['judgement_sent']
        ref = doc['headnote_sent']

        
        sentence_embeddings = get_sentence_embeddings(judgement)
        
        
        counts = compute_similarity_scores(sentence_embeddings, threshold=0.4)

        
        top_sentences = select_top_sentences(judgement, counts, top_percent=0.4)

       
        generated_summary = generate_summary(" ".join(top_sentences), tokenizer, model)

        
        scores = evaluate_summary(generated_summary, reference_summary, ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count += 1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

In [15]:
avg_scores_app4_1= evaluate_dataset_app4(dataset, tokenizer1, model1)
print(avg_scores_app4_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [17]:
avg_scores_app4_3= evaluate_dataset_app4(dataset, tokenizer3, model3)
print(avg_scores_app4_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [ ]:
avg_scores_app4_2= evaluate_dataset_app4(dataset, tokenizer2, model2)
print(avg_scores_app4_2)

1 2 

Input ids are automatically padded from 343 to 1024 to be a multiple of `config.attention_window`: 1024


3 

Input ids are automatically padded from 648 to 1024 to be a multiple of `config.attention_window`: 1024


4 

...

81 82 83 

Input ids are automatically padded from 466 to 1024 to be a multiple of `config.attention_window`: 1024


84 85 86 87 88 

Input ids are automatically padded from 838 to 1024 to be a multiple of `config.attention_window`: 1024


89 

# Checking direct abstractive approaches

In [12]:
def evaluate_dataset_app(dataset, tokenizer, model):
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
    }
    count=0

    for doc in dataset:
        reference_summary = " ".join(doc['headnote_sent'])
        judgement =   doc['judgement_sent']
        judgement_text = " ".join(judgement)
        ref = doc['headnote_sent']
        
        
        generated_summary = generate_summary(judgement_text, tokenizer, model)

        scores = evaluate_summary(generated_summary, reference_summary, ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count += 1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

In [11]:
avg_scores_app_1= evaluate_dataset_app(dataset, tokenizer1, model1)
print(avg_scores_app_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [48]:
avg_scores_app_3= evaluate_dataset_app(dataset, tokenizer3, model3)
print(avg_scores_app_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [10]:
avg_scores_app_2= evaluate_dataset_app(dataset, tokenizer2, model2)
print(avg_scores_app_2)

1 

Input ids are automatically padded from 734 to 1024 to be a multiple of `config.attention_window`: 1024


2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 

Input ids are automatically padded from 824 to 1024 to be a multiple of `config.attention_window`: 1024


42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 

...

340 341 342 343 344 345 346 347 348 349 

Input ids are automatically padded from 849 to 1024 to be a multiple of `config.attention_window`: 1024


350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 

Input ids are automatically padded from 928 to 1024 to be a multiple of `config.attention_window`: 1024


374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 
{'bleu': 0.12543655511121035, 'rouge1': 0.3600347997905578, 'rouge2': 0.13405223734838434, 'rougeL': 0.18540550261742236, 'precision': 0.04107370081818776, 'recall': 0.17404433032906286, 'f1': 0.06627566342545471}


# MODEL 5 - Legal RoBERTa

In [18]:
tokenizer5 = AutoTokenizer.from_pretrained("saibo/legal-roberta-base")
model5 = AutoModel.from_pretrained("saibo/legal-roberta-base").to(device)

def get_legalroberta_embeddings(sentences):
    embeddings = []
    
    for sentence in sentences:
        # Tokenize the sentence and convert to input IDs and attention masks
        inputs = tokenizer5(sentence, return_tensors='pt', padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()} 
        with torch.no_grad():
            outputs = model5(**inputs)
        
        # The embeddings are the output of the last hidden state
        # We take the mean of the token embeddings to get a sentence embedding
        sentence_embedding = outputs.last_hidden_state.mean(dim=1).cpu().squeeze()
        embeddings.append(sentence_embedding)

    return torch.stack(embeddings)
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity_scores(embeddings, threshold=0.4):
    sentence_count = len(embeddings)
    counts = [0] * sentence_count  # Initialize a count for each sentence

    for i in range(sentence_count):
        for j in range(sentence_count):
            if i != j:
                similarity = cosine_similarity(
                    embeddings[i].unsqueeze(0), embeddings[j].unsqueeze(0)
                )[0][0]
                if similarity > threshold:
                    counts[i] += 1
    return counts
def select_top_sentences(sentences, counts, top_percent=0.4):
    # Zip sentences with their counts and sort based on counts
    sorted_results = sorted(zip(sentences, counts), key=lambda x: x[1], reverse=True)
    num_sentences = int(top_percent * len(sorted_results))  # Top 40%
    top_sentences = [item[0] for item in sorted_results[:num_sentences]]  # Select sentences
    return top_sentences
def generate_summary_from_top_sentences(top_sentences):
    return ' '.join(top_sentences)


def evaluate_summary(generated_summary, reference_summary, ref):
    # Tokenization for BLEU score calculation
    reference = [nltk.word_tokenize(reference_summary)]
    hypothesis = nltk.sent_tokenize(generated_summary)
    ref_tokenized = [nltk.word_tokenize(sentence) for sentence in ref]

    # BLEU score using sacrebleu
    bleu_score = sacrebleu.corpus_bleu(hypothesis, ref_tokenized).score

    # ROUGE score calculation
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge.score(" ".join(reference_summary), " ".join(generated_summary))
    
    precision, recall, f1 = precision_recall_f1(" ".join(reference_summary), generated_summary)


    # You can remove or replace precision/recall/F1 with more appropriate metrics if needed
    return {
        "bleu": bleu_score,
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rouge2": rouge_scores['rouge2'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure,
        "precision":precision,
        "recall":recall,
        "f1":f1
    }


def evaluate_dataset_app5(dataset, tokenizer, model):
    count=0
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
      }

    for doc in dataset:
        
        judgement_text =  doc['judgement_sent']
        reference_summary = (doc['headnote_sent'])
        ref = doc['headnote_sent']

        
        sentence_embeddings = get_legalroberta_embeddings(judgement_text)
    
        
        counts = compute_similarity_scores(sentence_embeddings, threshold=0.4)
    
        
        top_sentences = select_top_sentences(judgement_text, counts, top_percent=0.4)
    
        
        filtered_summary = generate_summary_from_top_sentences(top_sentences)
        generated_summary = generate_summary(filtered_summary, tokenizer, model)
    
        
        scores = evaluate_summary((generated_summary), " ".join(reference_summary), ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count+=1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

Some weights of RobertaModel were not initialized from the model checkpoint at saibo/legal-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
avg_scores_app5_1= evaluate_dataset_app5(dataset, tokenizer1, model1)
print(avg_scores_app5_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [11]:
avg_scores_app5_3= evaluate_dataset_app5(dataset, tokenizer3, model3)
print(avg_scores_app5_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [19]:
avg_scores_app5_2= evaluate_dataset_app5(dataset, tokenizer2, model2)
print(avg_scores_app5_2)

1 2 

Input ids are automatically padded from 328 to 1024 to be a multiple of `config.attention_window`: 1024


3 

Input ids are automatically padded from 648 to 1024 to be a multiple of `config.attention_window`: 1024


4 

...

382 383 

Input ids are automatically padded from 967 to 1024 to be a multiple of `config.attention_window`: 1024


384 385 386 387 388 389 390 391 

Input ids are automatically padded from 793 to 1024 to be a multiple of `config.attention_window`: 1024


392 393 394 395 396 397 398 399 400 
{'bleu': 0.3371329635977834, 'rouge1': 0.6414228999409818, 'rouge2': 0.564796384618494, 'rougeL': 0.3777489386732128, 'precision': 0.04196750081171551, 'recall': 0.17587695296287123, 'f1': 0.06756997765024886}
